In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import warnings
from sklearn.preprocessing import StandardScaler

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Configuración de visualización
plt.style.use('classic')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
def cargar_y_preprocesar(ruta_archivo):
    print("=== Cargando y Preprocesando Datos ===")
    df = pd.read_csv(ruta_archivo)
    
    df['ano'] = pd.to_numeric(df['ano'], errors='coerce')
    df['n'] = pd.to_numeric(df['n'], errors='coerce')
    df['total'] = pd.to_numeric(df['total'], errors='coerce')
    df['dpto'] = df['dpto'].str.upper().str.strip()
    df['tasa_mortalidad'] = (df['n'] / df['total']) * 100000
    
    print(f"Dataset cargado con {df.shape[0]} filas y {df.shape[1]} columnas")
    return df

In [ ]:
def analizar_con_prophet(df):
    print("\n=== Análisis con Prophet ===")
    
    df_prophet = df.groupby('ano')['tasa_mortalidad'].mean().reset_index()
    df_prophet.columns = ['ds', 'y']
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'], format='%Y')
    
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        interval_width=0.95
    )
    
    model.fit(df_prophet)
    future_dates = model.make_future_dataframe(periods=5, freq='Y')
    forecast = model.predict(future_dates)
    
    plt.figure(figsize=(15, 10))
    plt.plot(df_prophet['ds'], df_prophet['y'], 'ko-', label='Datos históricos', alpha=0.6)
    plt.plot(forecast['ds'], forecast['yhat'], 'b-', label='Pronóstico', linewidth=2)
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'],
                     color='blue', alpha=0.2, label='Intervalo de confianza 95%')
    
    plt.title('Pronóstico de Mortalidad por Tiroides')
    plt.xlabel('Año')
    plt.ylabel('Tasa de Mortalidad (por 100,000 habitantes)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    fig2 = model.plot_components(forecast)
    plt.show()
    
    print("\nPronóstico para los próximos 5 años:")
    future_forecast = forecast[forecast['ds'] > df_prophet['ds'].max()]
    print("\nAño  |  Tasa Esperada  |  Intervalo de Confianza")
    print("-" * 50)
    for _, row in future_forecast.iterrows():
        print(f"{row['ds'].year}  |  {row['yhat']:.2f}  |  [{row['yhat_lower']:.2f}, {row['yhat_upper']:.2f}]")
    
    return model, forecast

In [ ]:
def analizar_tendencias_regionales_prophet(df):
    print("\n=== Análisis Regional con Prophet ===")
    
    top_dptos = df.groupby('dpto')['tasa_mortalidad'].mean().nlargest(5).index
    
    plt.figure(figsize=(15, 10))
    
    for dpto in top_dptos:
        df_dpto = df[df['dpto'] == dpto].copy()
        df_prophet = df_dpto.groupby('ano')['tasa_mortalidad'].mean().reset_index()
        
        # Verificar si hay suficientes datos
        if len(df_prophet) < 2:
            print(f"Insuficientes datos para {dpto}, omitiendo...")
            continue
            
        df_prophet.columns = ['ds', 'y']
        df_prophet['ds'] = pd.to_datetime(df_prophet['ds'], format='%Y')
        
        # Eliminar valores nulos
        df_prophet = df_prophet.dropna()
        
        try:
            model = Prophet(yearly_seasonality=True,
                          weekly_seasonality=False,
                          daily_seasonality=False)
            model.fit(df_prophet)
            
            future = model.make_future_dataframe(periods=5, freq='Y')
            forecast = model.predict(future)
            
            plt.plot(df_prophet['ds'], df_prophet['y'], 'o-', label=f'{dpto} (histórico)', alpha=0.6)
            plt.plot(forecast['ds'], forecast['yhat'], '--', label=f'{dpto} (pronóstico)')
        except Exception as e:
            print(f"Error procesando {dpto}: {str(e)}")
            continue
    
    plt.title('Pronóstico Regional de Mortalidad por Tiroides')
    plt.xlabel('Año')
    plt.ylabel('Tasa de Mortalidad (por 100,000 habitantes)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
def analizar_demografia(df):
    print("\n=== Análisis Demográfico ===")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    df['sexo'].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=ax1)
    ax1.set_title('Distribución por Sexo')
    
    df_edad = df.groupby('gru_edad')['n'].sum().sort_index()
    df_edad.plot(kind='bar', ax=ax2)
    ax2.set_title('Distribución por Grupo de Edad')
    ax2.set_xlabel('Grupo de Edad')
    ax2.set_ylabel('Número de Casos')
    plt.tight_layout()
    plt.show()

In [ ]:
def segmentar_departamentos(df):
    try:
        # Preparar datos
        dept_stats = df.groupby('dpto').agg({
            'tasa_mortalidad': ['mean', 'std', 'max']
        }).reset_index()
        
        dept_stats.columns = ['dpto_'] + ['_'.join(col).strip() for col in dept_stats.columns[1:]]
        
        # Manejar valores faltantes
        dept_stats = dept_stats.fillna({
            'tasa_mortalidad_mean': dept_stats['tasa_mortalidad_mean'].median(),
            'tasa_mortalidad_std': 0,
            'tasa_mortalidad_max': dept_stats['tasa_mortalidad_max'].median()
        })
        
        # Normalizar datos y aplicar clustering
        scaler = StandardScaler()
        X = scaler.fit_transform(dept_stats[['tasa_mortalidad_mean', 'tasa_mortalidad_std', 'tasa_mortalidad_max']])
        
        kmeans = KMeans(n_clusters=3, random_state=42)
        dept_stats['cluster'] = kmeans.fit_predict(X)
        
        # Crear gráfico mejorado
        plt.figure(figsize=(15, 10))
        colors = ['#FF9999', '#66B2FF', '#99FF99']
        markers = ['o', 's', '^']
        
        for i in range(3):
            cluster_data = dept_stats[dept_stats['cluster'] == i]
            plt.scatter(cluster_data['tasa_mortalidad_mean'],
                       cluster_data['tasa_mortalidad_std'],
                       c=colors[i],
                       marker=markers[i],
                       s=200,
                       label=f'Cluster {i+1}',
                       alpha=0.6)
            
            # Añadir etiquetas con mejor formato
            for _, row in cluster_data.iterrows():
                plt.annotate(row['dpto_'],
                           (row['tasa_mortalidad_mean'], row['tasa_mortalidad_std']),
                           xytext=(5, 5),
                           textcoords='offset points',
                           fontsize=9,
                           bbox=dict(facecolor='white', 
                                   edgecolor='gray',
                                   alpha=0.7,
                                   boxstyle='round,pad=0.3'))
        
        plt.title('Clusters de Departamentos por Mortalidad', size=14, pad=20)
        plt.xlabel('Tasa de Mortalidad Media', size=12)
        plt.ylabel('Desviación Estándar', size=12)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3, linestyle='--')
        
        # Ajustar límites y escala
        plt.xlim(0, dept_stats['tasa_mortalidad_mean'].max() * 1.1)
        plt.ylim(0, dept_stats['tasa_mortalidad_std'].max() * 1.1)
        
        plt.tight_layout()
        plt.show()
        
        # Imprimir resumen de clusters
        print("\nResumen de Clusters:")
        for i in range(3):
            cluster_data = dept_stats[dept_stats['cluster'] == i]
            print(f"\nCluster {i+1}:")
            print(f"Número de departamentos: {len(cluster_data)}")
            print(f"Tasa media: {cluster_data['tasa_mortalidad_mean'].mean():.2f}")
            print("Departamentos:", ", ".join(cluster_data['dpto_'].tolist()))
            
    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
def crear_visualizaciones(df):
    print("\n=== Visualizaciones Detalladas ===")
    
    # 1. Tasa de mortalidad por departamento últimos 10 años
    anos_max = df['ano'].max()
    df_reciente = df[df['ano'] > anos_max - 10]
    
    # Crear tabla pivote
    pivot_table = df_reciente.pivot_table(
        values='tasa_mortalidad',
        index='dpto',
        columns='ano',
        aggfunc='mean'
    )
    
    # Ordenar departamentos por promedio de tasa de mortalidad
    promedios = pivot_table.mean(axis=1)
    pivot_table = pivot_table.reindex(promedios.sort_values(ascending=False).index)
    
    plt.figure(figsize=(15, 12))
    
    # Crear heatmap con parámetros específicos
    sns.heatmap(
        pivot_table,
        cmap='Reds',
        annot=True,
        fmt='.0f',
        cbar_kws={
            'label': 'Tasa de Mortalidad por 100,000 habitantes',
            'orientation': 'horizontal'
        }
    )
    
    plt.title('Tasas de Mortalidad por Departamento\nÚltimos 10 años', pad=20)
    plt.xlabel('Año')
    plt.ylabel('Departamento')
    
    # Ajustar layout
    plt.tight_layout()
    plt.show()

    # 2. Distribución de Mortalidad por Grupo de Edad y Sexo
    plt.figure(figsize=(12, 6))
    pivot_edad_sexo = df.pivot_table(
        values='n',
        index='gru_edad',
        columns='sexo',
        aggfunc='sum'
    )
    pivot_edad_sexo.plot(kind='bar', stacked=True)
    plt.title('Distribución de Mortalidad por Grupo de Edad y Sexo')
    plt.xlabel('Grupo de Edad')
    plt.ylabel('Número de Casos')
    plt.legend(title='Sexo')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 3. Tasas de mortalidad por grupo de edad y vista detallada
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    sns.violinplot(data=df, x='gru_edad', y='tasa_mortalidad', ax=ax1, color='lightblue')
    ax1.set_title('Distribución de Tasas de Mortalidad por Grupo de Edad')
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
    
    # Vista detallada (sin outliers)
    threshold = df['tasa_mortalidad'].quantile(0.95)
    df_filtered = df[df['tasa_mortalidad'] <= threshold].copy()
    sns.violinplot(data=df_filtered, x='gru_edad', y='tasa_mortalidad', ax=ax2, color='lightblue')
    ax2.set_title('Vista Detallada (sin valores extremos)')
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)
    plt.tight_layout()
    plt.show()

    # 4 y 5. Análisis de mortalidad y población por departamento
    dept_stats = df.groupby('dpto').agg({
        'tasa_mortalidad': 'mean',
        'total': 'mean',
        'n': 'sum'
    }).reset_index()
    
    # Configuración de la figura con fondo gris claro
    plt.style.use('classic')
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 16))
    fig.patch.set_facecolor('#f0f0f0')
    
    # Gráfico para población > 20,000
    mask_high = dept_stats['total'] >= 20000
    high_data = dept_stats[mask_high]
    
    scatter1 = ax1.scatter(high_data['tasa_mortalidad'],
                          high_data['total'],
                          s=high_data['n'] * 2,
                          c=high_data['tasa_mortalidad'],
                          cmap='RdYlBu_r',
                          alpha=0.7,
                          edgecolor='white',
                          linewidth=1)
    
    # Etiquetas mejoradas para población alta
    for _, row in high_data.iterrows():
        label = ax1.annotate(row['dpto'], 
                           (row['tasa_mortalidad'], row['total']),
                           xytext=(10, 0),
                           textcoords='offset points',
                           bbox=dict(facecolor='white',
                                   edgecolor='gray',
                                   alpha=0.8,
                                   boxstyle='round,pad=0.3'),
                           horizontalalignment='left',
                           verticalalignment='center')
    
    # Gráfico para población < 20,000
    mask_low = dept_stats['total'] < 20000
    low_data = dept_stats[mask_low].sort_values('total', ascending=False)
    
    scatter2 = ax2.scatter(low_data['tasa_mortalidad'],
                          low_data['total'],
                          s=low_data['n'] * 3,
                          c=low_data['tasa_mortalidad'],
                          cmap='RdYlBu_r',
                          alpha=0.7,
                          edgecolor='white',
                          linewidth=1)
    
    # Etiquetas mejoradas para población baja
    for idx, row in low_data.iterrows():
        offset = 10 if idx % 2 == 0 else -10
        ha = 'left' if idx % 2 == 0 else 'right'
        label = ax2.annotate(row['dpto'],
                           (row['tasa_mortalidad'], row['total']),
                           xytext=(offset, 0),
                           textcoords='offset points',
                           bbox=dict(facecolor='white',
                                   edgecolor='gray',
                                   alpha=0.8,
                                   boxstyle='round,pad=0.3'),
                           horizontalalignment=ha,
                           verticalalignment='center')
    
    # Configuración común para ambos ejes
    for ax in [ax1, ax2]:
        ax.set_facecolor('white')
        ax.grid(True, alpha=0.3, linestyle='--', color='gray')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.tick_params(axis='both', which='major', labelsize=10)
    
    # Títulos y etiquetas
    ax1.set_title('Análisis de Mortalidad y Población por Departamento\n(Población > 20,000)', 
                 pad=20, size=12, weight='bold')
    ax2.set_title('Detalle de Departamentos con Población < 20,000', 
                 pad=20, size=12, weight='bold')
    
    for ax in [ax1, ax2]:
        ax.set_xlabel('Tasa de Mortalidad (por 100,000 habitantes)', size=10)
        ax.set_ylabel('Población Total', size=10)
    
    # Barra de color
    cbar = plt.colorbar(scatter1, ax=[ax1, ax2], 
                       label='Tasa de Mortalidad por 100k hab.',
                       orientation='vertical')
    cbar.ax.tick_params(labelsize=9)
    
    # Leyenda de tamaños
    legend_elements = [
        plt.scatter([], [], s=2000, c='gray', alpha=0.5, label='10,000 casos'),
        plt.scatter([], [], s=1000, c='gray', alpha=0.5, label='5,000 casos'),
        plt.scatter([], [], s=200, c='gray', alpha=0.5, label='1,000 casos')
    ]
    ax1.legend(handles=legend_elements, 
              title='Número de Casos',
              loc='upper right',
              bbox_to_anchor=(1.15, 1),
              fontsize=9,
              title_fontsize=10)
    
    # Ajustar límites
    ax1.set_xlim(4, 18)
    ax2.set_xlim(-2000, 10000)
    ax2.set_ylim(-5000, 20000)
    
    plt.tight_layout()
    plt.show()

    # 6. Evolución de la mortalidad por grupo de edad
    pivot_trend = df.pivot_table(
        values='n',
        index='ano',
        columns='gru_edad',
        aggfunc='sum'
    )
    plt.figure(figsize=(12, 6))
    pivot_trend.plot(kind='area', stacked=True)
    plt.title('Evolución de la Mortalidad por Grupo de Edad')
    plt.xlabel('Año')
    plt.ylabel('Número de Casos')
    plt.legend(title='Grupo de Edad', bbox_to_anchor=(1.05, 1))
    plt.tight_layout()
    plt.show()

    # 7. Matriz de correlación
    columnas_numericas = ['ano', 'n', 'total', 'tasa_mortalidad']
    df_num = df[columnas_numericas]
    corr_matrix = df_num.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, 
                annot=True,
                cmap='coolwarm',
                center=0,
                fmt='.2f',
                square=True)
    plt.title('Matriz de Correlación - Variables Numéricas')
    plt.tight_layout()
    plt.show()

In [ ]:
def main():
    df = cargar_y_preprocesar('datos.csv')
    
    model, forecast = analizar_con_prophet(df)
    analizar_tendencias_regionales_prophet(df)
    analizar_demografia(df)
    segmentar_departamentos(df)
    crear_visualizaciones(df)
    
    print("\nAnálisis completo")

if __name__ == "__main__":
    main()